目標: 
一個src，
- 可以前處理資料:
1. 補缺少的日期、hr (不含weekday)

- 可以在特定weekday底下，將所有站別分群，分群的邏輯如下:
1. 計算特定站別在每個weekday、每個Hr底下的出租量的Pr50，然後計算其三小時的移動平均
2. 第1點的移動平均建一個線性回歸，看其租借量的趨勢，得到slope & intersection
3. 上述的slope & intersect作為分群的依據


In [1]:
import pandas as pd
import numpy as np
from dataclasses import asdict, dataclass, field
from typing import Optional, List, Union
import tqdm
import json
# plot
import matplotlib.pyplot as plt
import seaborn as sns
# model
from scipy import stats
from sklearn import cluster


In [6]:
RENT_COLS = ['station', 'Generation', 'sno', 'time', 'date', 'Hr', 'rent_count', 'weekday']
WEEKDAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
data = pd.read_csv('./data/df_Final.csv', usecols=RENT_COLS)
data['date'] = pd.to_datetime(data['date'])
data_rentCnt = data.pivot_table(
    index=['station', 'sno', 'date', 'weekday'],
    columns='Hr',
    values='rent_count',
    aggfunc='sum',
).reset_index()

In [7]:
class StationHrDes:
    COLS_REQ = ['station', 'sno', 'date', 'weekday'] + [i for i in range(24)]
    WEEKDAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    def __init__(self, df_station: Optional[pd.DataFrame], sno: Optional[int] = None, hr_ma_windows=3):
        assert len(col_miss := (set(self.COLS_REQ) - set(df_station.columns))) == 0, f"columns requirements: 'station', 'sno', 'date', 'weekday', 0,1,2,...,23 (hr); Missing cols: {col_miss}"
        # 計算每個weekday、hr的逐三小時移動平均
        self.MA_hr_pr50 = self.fill_miss_days(df_station)\
            .groupby('weekday')\
            .apply(lambda x: self.weekday_hr_ma(x, windows=hr_ma_windows))
        #照星期順序排
        self.MA_hr_pr50 = self.MA_hr_pr50.loc[[w for w in self.WEEKDAYS if w in self.MA_hr_pr50.index]]

        self.lr_des = self.lr_des_perttyFormat(self.MA_hr_pr50.apply(self.scipy_lr, axis=1)).assign(sno=sno)

        self.sno = sno

    def fill_miss_days(self, df):
        """
        輸入一個df，依據'date'欄位找出缺漏的日期，為該日期的租借量補上nan
        """
        d_min, d_max = df['date'].apply(['min', 'max'])
        days = pd.Series(index=pd.date_range(d_min, d_max, freq='d'), dtype=float)
        ans = pd.concat([df.set_index('date'), days], axis=1).iloc[:, :-1]
        ans.fillna(0, inplace=True)
        ans.index.name = 'date'
        return ans

    def weekday_hr_ma(self, df, windows=3, enough_samples=5):
        """
        輸入只有一個weekday的subset。回傳各小時的出租量的Pr50的近三小時移動平均。
        目標是用來判斷是否為高峰。
        - 如果某小時的樣本少於enough_samples(default=5)就設為nan
        """
        hr_Pr50 = df[np.arange(24)].quantile(.5)
        hr_cnt = (~df[np.arange(24)].isna()).sum()
        ans = pd.concat([hr_Pr50.iloc[-2:], hr_Pr50])\
                .rolling(windows)\
                .mean()\
                .iloc[2:] # 0點的移動平均要從10, 11, 12計算
        enough_mask = hr_cnt < enough_samples  # 如果樣本少於enough_samples(default=5)就不採用
        ans.loc[enough_mask] = 0
        ans.index.name = 'weekday'
        return ans

    def scipy_lr(self, ser, return_line=False):
        slope, intersect, rscore, pvalue, serr = stats.linregress(np.arange(len(ser)), ser)
        if not return_line:
            return slope, intersect
        return np.arange(len(ser)) * slope + intersect

    def lr_des_perttyFormat(self, lr_des):
        return pd.DataFrame([[i[0] for i in lr_des], [i[1] for i in lr_des]], columns=lr_des.index, index=['slope', 'intersect']).T


In [39]:
@dataclass
class StationsLrHolder:
    stations_info: List[StationHrDes] = field(default_factory=list)
    LR_data: Optional[pd.DataFrame] = None

    def add_one_station(self, des: Union[pd.DataFrame, StationHrDes], sno: Optional[int] = None) -> None:
        if isinstance(des, pd.DataFrame):
            self.stations_info.append(self.df_2_StationHrDes(des, sno))
        elif isinstance(des, StationHrDes):
            self.stations_info.append(des)
        else:
            raise TypeError("wrong type of des, it should be either pd.DataFrame or StationHrDes.")

    def df_2_StationHrDes(self, df_station: pd.DataFrame, sno: Optional[int] = None) -> StationHrDes:
        return StationHrDes(df_station, sno)

    def update_LR_data(self) -> None:
        holder = []
        for info in self.stations_info:
            holder.append(info.lr_des)
        self.LR_data = pd.concat(holder).reset_index()


def weekday_classification(LR_data: pd.DataFrame, weekday: str, n_cluster=5, return_model=False, return_LR=False):
    if set(['slope', 'intersect', 'weekday']) - set(LR_data.columns):
        raise KeyError("'slope', 'intersect', 'weekday should be in LR_data.")
    sub = LR_data.query(" weekday==@weekday ")
    m = cluster.KMeans(n_cluster)
    sub_std = sub[['slope', 'intersect']].apply(lambda x: (x - x.min()) / (x.max() - x.min()), axis=0)
    labels = m.fit_predict(sub_std[['slope', 'intersect']].values)
    ans: pd.Series = pd.Series(labels, index=sub['sno'])
    ans.name = 'cluster'
    if return_LR:
        return pd.concat([sub.set_index('sno'), ans], axis=1)
    if return_model:
        return ans, m
    return ans

In [9]:
stations = StationsLrHolder()
iters = tqdm.tqdm(data_rentCnt.groupby('sno'), total=data_rentCnt['sno'].nunique())
for k, g in iters:
    stations.add_one_station(g, sno=k)   
stations.update_LR_data()
stations.LR_data.query(" weekday=='Monday' ")

100%|██████████| 1418/1418 [00:27<00:00, 51.99it/s]


,weekday,slope,intersect,sno
0,Monday,2.626304,1.985000,1
7,Monday,1.765000,4.348333,2
14,Monday,0.264783,0.163333,4
21,Monday,1.624783,-3.060000,5
28,Monday,1.227391,-3.906667,6
...,...,...,...,...
9864,Monday,0.078696,0.136667,500119086
9871,Monday,0.203913,0.196667,500119087
9878,Monday,0.217826,0.786667,500119088
9885,Monday,0.082391,0.323333,500119089


In [62]:
for wd in WEEKDAYS:
    weekday_lr = weekday_classification(stations.LR_data, weekday=wd, return_LR=True)
    with open(f'./LR/LR_{wd}.json', 'w') as f:
        json.dump(weekday_lr.drop('weekday', axis=1).to_dict(orient='index'), f, indent=2)


{1: {'slope': 2.5265217391304353,
  'intersect': -1.1800000000000068,
  'cluster': 4},
 2: {'slope': 1.3693478260869565,
  'intersect': 2.2733333333333334,
  'cluster': 1},
 4: {'slope': 0.14304347826086955,
  'intersect': 0.2300000000000002,
  'cluster': 0},
 5: {'slope': 0.8047826086956522,
  'intersect': 0.7033333333333331,
  'cluster': 3},
 6: {'slope': 1.0821739130434782,
  'intersect': -2.6950000000000003,
  'cluster': 3},
 7: {'slope': 1.118695652173913,
  'intersect': -0.2400000000000002,
  'cluster': 3},
 8: {'slope': 0.2806521739130435,
  'intersect': 0.20999999999999996,
  'cluster': 0},
 9: {'slope': 0.1739130434782609,
  'intersect': 1.583333333333333,
  'cluster': 2},
 10: {'slope': 0.5660869565217392,
  'intersect': 2.4899999999999984,
  'cluster': 1},
 11: {'slope': 0.5006521739130435,
  'intersect': 3.5549999999999997,
  'cluster': 1},
 12: {'slope': 0.6645652173913045,
  'intersect': 1.4199999999999982,
  'cluster': 1},
 14: {'slope': 0.40282608695652167,
  'intersect

In [41]:
labels

,weekday,slope,intersect,cluster
sno,,,,
1,Sunday,2.526522,-1.180000,2
2,Sunday,1.369348,2.273333,3
4,Sunday,0.143043,0.230000,1
5,Sunday,0.804783,0.703333,4
6,Sunday,1.082174,-2.695000,4
...,...,...,...,...
500119086,Sunday,0.005652,-0.023333,1
500119087,Sunday,0.128261,-0.100000,1
500119088,Sunday,0.121304,0.063333,1
